In [1]:
import json

from doml_mc.model.doml_model import parse_doml_model

In [2]:
from doml_mc.intermediate_model.metamodel import parse_metamodel
import yaml
with open("assets/doml_meta.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)

In [3]:
with open("POSIDONIA.doml") as jsonf:
    doc = json.load(jsonf)

doml_model = parse_doml_model(doc, mm)


In [4]:
from doml_mc.intermediate_model.doml_model2im import doml_model_to_im
im = doml_model_to_im(doml_model)

In [5]:
im["prodevelop_edi_procesos_container"]

DOMLElement(name='prodevelop_edi_procesos_container', type='infrastructure_Container', attributes={'commons_DOMLElement::name': 'prodevelop_edi_procesos_container'}, associations={'infrastructure_Container::host': {'Plataforma EDI'}, 'infrastructure_Container::generatedFrom': {'prodevelop_edi_procesos_image'}, 'infrastructure_ComputingNode::ifaces': set()})

In [6]:
from z3 import Solver

from doml_mc.z3.metamodel_encoding import (
    def_association_rel_and_assert_types,
    def_attribute_rel_and_assert_types,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from doml_mc.z3.im_encoding import (
    assert_im_associations,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from doml_mc.z3.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm)
assoc_sort, assoc = mk_association_sort_dict(mm)
attr_sort, attr = mk_attribute_sort_dict(mm)
elem_sort, elem = mk_elem_sort_dict(im)
ss_sort, ss = mk_stringsym_sort_dict(im, mm)
AData = mk_adata_sort(ss_sort)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
assoc_rel = def_association_rel_and_assert_types(
    mm,
    solver,
    assoc_sort,
    assoc,
    class_,
    elem_class_f,
    elem_sort
)
assert_im_associations(
    assoc_rel,
    solver,
    im,
    mm,
    elem,
    assoc
)
attr_rel = def_attribute_rel_and_assert_types(
    mm,
    solver,
    attr_sort,
    attr,
    class_,
    elem_class_f,
    elem_sort,
    AData,
    ss
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)

In [7]:
solver.check()

sat